**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Permissive Policy

*permissive* allows everything

https://kubernetes.io/docs/concepts/policy/pod-security-policy

Assume, we want to start a system service on a host port


In [1]:
kubectl get psp

NAME          PRIV    CAPS   SELINUX    RUNASUSER   FSGROUP    SUPGROUP   READONLYROOTFS   VOLUMES
restrictive   false   *      RunAsAny   RunAsAny    RunAsAny   RunAsAny   false            configMap,downwardAPI,emptyDir,persistentVolumeClaim,secret,projected


In [14]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-hostnetwork-deployment
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
      hostNetwork: true ### try to access the host network
EOF

deployment.apps/nginx-hostnetwork-deployment created


In [16]:
kubectl get all

NAME                                              READY   STATUS    RESTARTS   AGE
pod/nginx-hostnetwork-deployment-db7654df-88wgs   1/1     Running   0          14s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   171m

NAME                                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-hostnetwork-deployment   1/1     1            1           14s

NAME                                                    DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-hostnetwork-deployment-db7654df   1         1         1       14s


In [17]:
RS=$(kubectl get  rs | awk '/nginx-hostnetwork-deployment/{print $1}' | head -1)
echo $RS

nginx-hostnetwork-deployment-db7654df


In [18]:
kubectl describe rs $RS | grep --color=always -E   '($|FailedCreate|Host.*)'

Name:           nginx-hostnetwork-deployment-db7654df
Namespace:      default
Selector:       app=nginx,pod-template-hash=db7654df
Labels:         app=nginx
                pod-template-hash=db7654df
Annotations:    deployment.kubernetes.io/desired-replicas: 1
                deployment.kubernetes.io/max-replicas: 2
                deployment.kubernetes.io/revision: 1
Controlled By:  Deployment/nginx-hostnetwork-deployment
Replicas:       1 current / 1 desired
Pods Status:    1 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  app=nginx
           pod-template-hash=db7654df
  Containers:
   nginx:
    Image:        nginx
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Events:
  Type    Reason            Age   From                   Message
  ----    ------            ----  ----                   -------
  Normal  SuccessfulCreate  25s   replicaset-controller  Created pod: nginx-hostnetwork-

**Forbidden by default**

Works as intended, let us open the box of pandora

## Permissive Policy

In [6]:
kubectl create -f - <<EOF
apiVersion: policy/v1beta1
kind: PodSecurityPolicy
metadata:
  name: permissive
spec:
  privileged: true
  hostNetwork: true
  hostIPC: true
  hostPID: true
  seLinux:
    rule: RunAsAny
  supplementalGroups:
    rule: RunAsAny
  runAsUser:
    rule: RunAsAny
  fsGroup:
    rule: RunAsAny
  hostPorts:
  - min: 0
    max: 65535
  volumes:
  - '*'
EOF

podsecuritypolicy.policy/permissive created


In [7]:
kubectl apply -f - <<EOF
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: psp-permissive
rules:
- apiGroups:
  - extensions
  resources:
  - podsecuritypolicies
  resourceNames:
  - permissive
  verbs:
  - use
EOF

clusterrole.rbac.authorization.k8s.io/psp-permissive created


In [11]:
kubectl get clusterrole

NAME                                                                   CREATED AT
admin                                                                  2021-01-28T10:00:32Z
cluster-admin                                                          2021-01-28T10:00:32Z
edit                                                                   2021-01-28T10:00:32Z
kubeadm:get-nodes                                                      2021-01-28T10:00:35Z
psp-permissive                                                         2021-01-28T12:48:42Z
psp-restrictive                                                        2021-01-28T11:26:03Z
system:aggregate-to-admin                                              2021-01-28T10:00:32Z
system:aggregate-to-edit                                               2021-01-28T10:00:32Z
system:aggregate-to-view                                               2021-01-28T10:00:32Z
system:auth-delegator                                                  2021-01-28T10:00:32

In [8]:
kubectl create -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1beta1
kind: RoleBinding
metadata:
  name: permissive
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: psp-permissive
subjects:
- kind: ServiceAccount
  name: default
  namespace: default
EOF

rolebinding.rbac.authorization.k8s.io/permissive created


In [19]:
echo goto http://$(hostname -i)/

goto http://192.168.49.2/


In [ ]:
kubectl delete deployment nginx-hostnetwork-deployment

## Recreate


go back and recreate the nginx-hostnetwork-deployment


In [20]:
kubectl get all

NAME                                              READY   STATUS    RESTARTS   AGE
pod/nginx-hostnetwork-deployment-db7654df-88wgs   1/1     Running   0          2m46s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   173m

NAME                                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-hostnetwork-deployment   1/1     1            1           2m46s

NAME                                                    DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-hostnetwork-deployment-db7654df   1         1         1       2m46s


## Which policy has been applied

how can we detect, which policy is in use for a given pod 

In [21]:
POD=$(kubectl get  pod | awk '/nginx-hostnetwork-deployment/{print $1}' | head -1)
echo $POD

nginx-hostnetwork-deployment-db7654df-88wgs


In [22]:
kubectl get pod $POD -o jsonpath='{.metadata.annotations.kubernetes\.io/psp}'

permissive

In [23]:
kubectl get pods -o jsonpath='{.items[*].metadata.annotations.kubernetes\.io/psp}'

permissive

## Can we still create the *normal* deployment

In [24]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-other-deployment
  namespace: default
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
EOF

deployment.apps/nginx-other-deployment created


In [25]:
kubectl get all

NAME                                              READY   STATUS    RESTARTS   AGE
pod/nginx-hostnetwork-deployment-db7654df-88wgs   1/1     Running   0          10m
pod/nginx-other-deployment-6799fc88d8-27zdq       1/1     Running   0          5s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   3h1m

NAME                                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-hostnetwork-deployment   1/1     1            1           10m
deployment.apps/nginx-other-deployment         1/1     1            1           6s

NAME                                                    DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-hostnetwork-deployment-db7654df   1         1         1       10m
replicaset.apps/nginx-other-deployment-6799fc88d8       1         1         1       6s


In [26]:
POD=$(kubectl get  pod | awk '/nginx-other-deployment/{print $1}' | head -1)
echo $POD

nginx-other-deployment-6799fc88d8-27zdq


In [27]:
kubectl get pods -o jsonpath="{range .items[*]}{.metadata.name}:  {.metadata.annotations.kubernetes\.io/psp}  
{end}"


nginx-hostnetwork-deployment-db7654df-88wgs:  permissive  
nginx-other-deployment-6799fc88d8-27zdq:  permissive  


## Conclusion

* don't mess up the psp, clusterroles and bindings
* better one role per account
* don't work with more than one rolebinding

## Next Stop

*Kubernetes Roles*